### Week 11 - changing pivot table to function and adding a heat map

I'm mostly working in the final notebook file as I'm cleaning and compiling. I did discover this week that pivot tables can easily be converted to heatmaps which I think would be very effective for my data set. I have been working to drop values lower than 10 from the dataset to avoid cross tab tables with insiginifcant numbers. <br>
Working functions below transferred as back up from final processing, datasets won't run here - input df is 65 0000 rows long

In [2]:
# Group data frame and drop rows if not meeting minimum threshold
df = swecalc_df
groupby = df.groupby(["zbin", "Aspect"],as_index=False).agg({"water_vol":["count","sum"]})
# drop rows which have a count of less than 10
groupby = groupby[groupby["water_vol"]["count"]>10]
# change column names
groupby.columns = ["zbin", "Aspect", "Count", "water_vol"]


summary = pd.crosstab(
      index=groupby['zbin'], columns=groupby['Aspect'],aggfunc="sum",
     values=groupby["water_vol"],margins=True,margins_name="Total").fillna(0).round(0)
all_aspects = ["North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest"]

hmap = pd.crosstab(
      index=groupby['zbin'], columns=groupby['Aspect'],aggfunc="sum",
     values=groupby["water_vol"],margins=False).fillna(0).round(0)

hmap = hmap[all_aspects]
plt.figure(figsize=(16,5))
#fmt value changes the formatting of the final numbers
sns.heatmap(hmap,
            cmap="YlGnBu", annot=True, cbar=True,fmt='g')
plt.title()
plt.show()

NameError: name 'swecalc_df' is not defined

Old function below - this one is hard to remove small values, as first need to group by both Aspect and elevation to drop the values which are less than 10

In [3]:
def pivot_table(df,value_col, agg_func,basin_name,wshed,phase):
    print("processing.. pivot table")
    #drop values in data frame where the SWE is less than 100, or equals flat aspect
    #df = df[df["Aspect"]!="Flat"]
    sum = pd.crosstab(
      index=df['zbin'], columns=df['Aspect'],
      values=df[value_col], aggfunc=agg_func, margins=True,margins_name="Total").fillna(0).round(0)
    #sum = sum[sum["zbin"]["Aspect"]>100]
    #sum[(sum["Aspect"]>100).any(axis=1)]
    #loop to order the aspects and drop if the value does not exist
    all_aspects = ["North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest"]
    #rearrange order of aspect columns for consistent pivot table
    sum = sum[all_aspects]
    sns.heatmap(sum,
            cmap="YlGnBu", annot=True, cbar=False)
    display(sum)
    sum.to_csv("outputs/pivot_{}_{}_{}_{}.csv".format(wshed,basin_name,phase,value_col),index=False)
    return sum

## Elevation graph including all sub-basins to function

back up of working code here, moving to function on the main aco clean up notebook

In [1]:
import matplotlib.ticker as ticker
#graph for SWE by elevation 
df = zbin_append
df = df.reset_index()
display(df)

#set colour and style for grpah
sns.set_style("darkgrid")
palette = sns.color_palette("YlGnBu", 10)

#create figure
fig, ax = plt.subplots(figsize=(10,6))
fig.set_tight_layout(True)

#loop through basins, generate line based on its water volume
index = 4
for i in df["Basin"].unique():
    print(i)
    line = df[df["Basin"]== i]
    line_plot = ax.plot(line["zbin"], line["water_vol"], "-o", color=palette[index],label= i)
    ax.fill_between(line["zbin"], line["lower_wv"],line["upper_wv"], alpha=0.2, color=palette[index])
    plt.legend()
    index = index + 2 #increase colour index for next line by 2

# set axis labels
ax.set_xlabel('Elevation [m, ellipsoid]')
ax.set_ylabel("Water Storage [$m^{3}$]")

#set x tick spacing
tick_spacing = 100
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

#set the axis range of the y
max_y = df["water_vol"].max().round(decimals=-6)+300000
yticks = (np.arange(0,max_y, (max_y/10).round(-5))).astype(int)
plt.yticks(yticks)

#generate the formatted y tick labels
y_labels = []
for i in yticks:
    value = format (i, ',d')
    y_labels.append(value)
display(y_labels)
ax.set_yticklabels(y_labels)      
ax.set_title("Total Water Storage: {}, Phase {}".format(wshed, phase[2:]), fontsize = 15)
       

    
plt.show()

NameError: name 'zbin_append' is not defined

## Plotly polar graph to function
This function is for spitting out graph for the individual phase

In [2]:
#polar plot by aspect
#convert colors for plotly formatting
pal = sns.color_palette("viridis")
clist = pal.as_hex()

#hex to rgb testing
import matplotlib.colors as colors

def hex_to_rgb(clist):
    #converting seaborne colors to hex value
    rgb_list = []
    new_list = []
    #converting the list of hex values to rgb - this changes decimal to number
    for h in clist:
        rgb = colors.hex2color(h)
        color = list([int(255*x) for x in rgb])
    #this line is appending a transparency value
        color.append(0.3)
        tuplecolor = tuple(color)
        rgb_list.append(tuplecolor)
    
    for x in rgb_list:
        new_list.append("rgba" + str(x))
    return(new_list)

color_error = hex_to_rgb(clist) #need to input a list of hex values here
display(color_error)

df = asp_append.reset_index()
display(df)
df = df[df["Asp"]>0]


title = "SWE [mm]: {}, phase {}".format(wshed,phase[2:])
swe_max = round((df["swe_hm"].max()+100),-2)
print(swe_max)
sns.set_style("darkgrid")
#set font
font_dict = dict(family="Arial", size = 26, color = "black")

#set the layout parameters for figure
layout = go.Layout(polar=dict(
angularaxis = dict(thetaunit="degrees",
            dtick=45,
             rotation=90,
             direction='clockwise',
             tickmode="array",
             tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
             ticktext=["N", "NE", "E", "SE", "S", "SW", "W", "NW"])
         ,radialaxis=dict(range=[1,swe_max]))
     )

fig = go.Figure(layout=layout)

fig.update_layout(
    title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

#set the layout parameters for figure
layout = go.Layout(polar=dict(
angularaxis = dict(thetaunit="degrees",
            dtick=45,
             rotation=90,
             direction='clockwise',
             tickmode="array",
             tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
             ticktext=["N", "NE", "E", "SE", "S", "SW", "W", "NW"])
         ,radialaxis=dict(range=[1,swe_max]))
     )

fig = go.Figure(layout=layout)

fig.update_layout(
    title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

def add_trace(phase, index, name):
    fig.add_trace(go.Scatterpolar(
                     r = line['swe_hm'],
                     theta = line['Asp'],
                     mode = "lines",
                    showlegend=False,
                     line_color = "rgba(44, 160, 44, 0)"
            ))
    fig.add_trace(go.Scatterpolar(
            r = line['swe_mm'],
            theta = line['Asp'],
            mode = 'lines',
            fill="tonext",
            name=name,
            line_color = clist[index]
            ))
                      
    fig.add_trace(go.Scatterpolar(
            r = line['swe_lm'],
            theta = line['Asp'],
            mode = 'none',
            fill="tonext",
            showlegend=False,
            fillcolor = color_error[index]
            )
                    )

#WAnt to loop through the basins adn produce the line accordingly by calling function
index = 0
for i in df["Basin"].unique():
        print(i)
        line = df[df["Basin"]== i]
        add_trace(line, index, i)
        index = index + 2
fig.show()

NameError: name 'sns' is not defined